In [1]:
import numpy as np
import pandas as pd

In [2]:
# Encodings with -1 and without one hot encoding
train_1 = pd.read_csv("train_1encoded.csv")
train_1['SettlementProcess'].replace({'Card':0, 'Bank':1, 'Check':2, 'Electronic':3}, inplace=True)
test_1  = pd.read_csv("test_1encoded.csv")
test_1['SettlementProcess'].replace({'Card':0, 'Bank':1, 'Check':2, 'Electronic':3}, inplace=True)

# Encodings without -1 and with one hot encoding
train_2 = pd.read_csv("train_encoded_noh.csv")
train_2['SettlementProcess'].replace({'Card':0, 'Bank':1, 'Check':2, 'Electronic':3}, inplace=True)
test_2  = pd.read_csv("test_encoded_noh.csv")
test_2['SettlementProcess'].replace({'Card':0, 'Bank':1, 'Check':2, 'Electronic':3}, inplace=True)

# Trivial encoding based on prev without 1 and one hot encoding is done
train_3 = pd.read_csv("train_encoded.csv")
test_3  = pd.read_csv("test_encoded.csv")

# Introduce few new features train_1  ----->

train_4 = train_1.copy()
train_4['G/Q'] = train_4['GrandPayment']/train_4['QuarterlyPayment']
train_4['G/S'] = train_4['GrandPayment']/(train_4['ServiceSpan']+3)
train_4['Q/S'] = train_4['QuarterlyPayment']/(train_4['ServiceSpan']+3)
train_4['Services_add'] = train_4['MobileService']+train_4['4GService']+ train_4['CyberProtection']+ train_4['HardwareSupport']+train_4['TechnicalAssistance'] + train_4['FilmSubscription']
train_4['SAMT'] = train_4['sex']*(1-train_4['Aged'])*(1-train_4['Married']) +train_4['TotalDependents']

test_4 = test_1.copy()
test_4['G/Q'] = test_4['GrandPayment']/test_4['QuarterlyPayment']
test_4['G/S'] = test_4['GrandPayment']/(test_4['ServiceSpan'] +3)
test_4['Q/S'] = test_4['QuarterlyPayment']/(test_4['ServiceSpan']+3)
test_4['Services_add'] = test_4['MobileService']+test_4['4GService']+ test_4['CyberProtection']+ test_4['HardwareSupport']+test_4['TechnicalAssistance'] + test_4['FilmSubscription']
test_4['SAMT'] = test_4['sex']*(1-test_4['Aged'])*(1-test_4['Married']) +test_4['TotalDependents']

# Introduce few new features train_2  ----->

train_5 = train_2.copy()
train_5['G/Q'] = train_5['GrandPayment']/train_5['QuarterlyPayment']
train_5['G/S'] = train_5['GrandPayment']/(train_5['ServiceSpan'] +1)
train_5['Q/S'] = train_5['QuarterlyPayment']/(train_5['ServiceSpan']+1)
train_5['Services_add']= train_5['MobileService']+train_5['4GService']+ train_5['CyberProtection']+ train_5['HardwareSupport']+train_5['TechnicalAssistance'] + train_5['FilmSubscription']
train_5['SAMT'] = train_5['sex']*(1-train_5['Aged'])*(1-train_5['Married']) +train_5['TotalDependents']

test_5 = test_2.copy()
test_5['G/Q'] = test_5['GrandPayment']/test_5['QuarterlyPayment']
test_5['G/S'] = test_5['GrandPayment']/(test_5['ServiceSpan']+1)
test_5['Q/S'] = test_5['QuarterlyPayment']/(test_5['ServiceSpan'] +1)
test_5['Services_add'] = test_5['MobileService']+test_5['4GService']+ test_5['CyberProtection']+ test_5['HardwareSupport']+test_5['TechnicalAssistance'] + test_5['FilmSubscription']
test_5['SAMT'] = test_5['sex']*(1-test_5['Aged'])*(1-test_5['Married']) +test_5['TotalDependents']

# Introduce few new features train_3  ----->

train_6 = train_3.copy()
train_6['G/Q'] = train_6['GrandPayment']/train_6['QuarterlyPayment']
train_6['G/S'] = train_6['GrandPayment']/(train_6['ServiceSpan']+1)
train_6['Q/S'] = train_6['QuarterlyPayment']/(train_6['ServiceSpan']+1)
# train_6['Services_add'] = train_6['MobileService']+train_6['4GService']+ train_6['CyberProtection']+ train_6['HardwareSupport']+train_6['TechnicalAssistance'] + train_6['FilmSubscription']
# train_6['SAMT'] = train_6['sex']*(1-train_6['Aged'])*(1-train_6['Married']) +train_6['TotalDependents']

test_6 = test_3.copy()
test_6['G/Q'] = test_6['GrandPayment']/test_6['QuarterlyPayment']
test_6['G/S'] = test_6['GrandPayment']/(test_6['ServiceSpan']+1)
test_6['Q/S'] = test_6['QuarterlyPayment']/(test_6['ServiceSpan']+1)
# test_6['Services_add'] = test_6['MobileService']+test_6['4GService']+ test_6['CyberProtection']+ test_6['HardwareSupport']+test_6['TechnicalAssistance'] + test_6['FilmSubscription']
# test_6['SAMT'] = test_6['sex']*(1-test_6['Aged'])*(1-test_6['Married']) +test_6['TotalDependents']


In [3]:
### General ###
import os
import sys
import copy
import pickle
import random
import warnings
warnings.filterwarnings("ignore")
sys.path.append("./tfms")

### Data Wrangling ###
import numpy as np
import pandas as pd
from scipy import stats
from gauss_rank_scaler import GaussRankScaler

### Data Visualization ###
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use("fivethirtyeight")

### Machine Learning ###
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import QuantileTransformer
from sklearn.feature_selection import VarianceThreshold
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

### Make prettier the prints ###
from colorama import Fore
c_ = Fore.CYAN
m_ = Fore.MAGENTA
r_ = Fore.RED
b_ = Fore.BLUE
y_ = Fore.YELLOW
g_ = Fore.GREEN

In [4]:
from sklearn.tree import DecisionTreeClassifier 
from sklearn.metrics import accuracy_score, f1_score
from sklearn.ensemble import GradientBoostingClassifier

from catboost import CatBoostRegressor , CatBoostClassifier

In [5]:
import xgboost as xgb

In [6]:
seed = 42

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
set_seed(seed)

In [7]:
scale = "rankgauss"   # boxcox , norm ,minmax , rankgauss
variance_threshould = None
decompo = "no" #"PCA"
ncompo = 9

In [8]:
def get_dummies(df,dum_cols,val = 0):
    if val == 0:
        return df
    return pd.get_dummies(df, prefix=None, prefix_sep='_', dummy_na=False, columns=dum_cols)

def normalize(df):
    return (df - df.mean(0) )/df.std(0)

In [9]:
train_df = train_2.copy()
test_df  = test_2.copy()

In [10]:
train_features = train_df.drop(['CustomerAttrition','ID'],axis=1)
train_targets_scored = train_df['CustomerAttrition']

test_features = test_df.drop(['ID'],axis=1)
sample_submission = pd.read_csv('Sample Submission.csv')

concat = pd.concat([train_features,test_features],axis=0)

create_dummy = 0                #0 if dont want dummies
dum_cols = ['age', 'experience', 'married', 'house_ownership',
       'car_ownership', 'profession', 'city', 'state', 'current_job_years',
       'current_house_years']

concat = get_dummies(concat,dum_cols,create_dummy)
# norm_concat = normalize(concat)

train_features = concat[:len(train_features)]
test_features  = concat[len(train_features):]

In [11]:
data_all = concat.copy()
cols_numeric = [feat for feat in list(data_all.columns)]
if variance_threshould:
    mask = (data_all[cols_numeric].var() >= variance_threshould).values
    data_all = data_all[cols_numeric].loc[:, mask]
    cols_numeric = [feat for feat in list(data_all.columns)]
data_all.reset_index(inplace=True)

In [12]:
data_all.isin([np.inf, -np.inf]).sum(axis=0)

index                  0
sex                    0
Aged                   0
Married                0
TotalDependents        0
ServiceSpan            0
MobileService          0
4GService              0
CyberProtection        0
HardwareSupport        0
TechnicalAssistance    0
FilmSubscription       0
SettlementProcess      0
QuarterlyPayment       0
GrandPayment           0
dtype: int64

In [13]:
def scale_minmax(col):
    return (col - col.min()) / (col.max() - col.min())

def scale_norm(col):
    return (col - col.mean()) / col.std()

if scale == "boxcox":
    print(b_, "boxcox")
    data_all[cols_numeric] = data_all[cols_numeric].apply(scale_minmax, axis = 0)
    trans = []
    for feat in cols_numeric:
        trans_var, lambda_var = stats.boxcox(data_all[feat].dropna() + 1)
        trans.append(scale_minmax(trans_var))
    data_all[cols_numeric] = np.asarray(trans).T
    
elif scale == "norm":
    print(b_, "norm")
    data_all[cols_numeric] = data_all[cols_numeric].apply(scale_norm, axis = 0)
    
elif scale == "minmax":
    print(b_, "minmax")
    data_all[cols_numeric] = data_all[cols_numeric].apply(scale_minmax, axis = 0)
    
elif scale == "rankgauss":
    ### Rank Gauss ###
    print(b_, "Rank Gauss")
    scaler = GaussRankScaler()
    data_all[cols_numeric] = scaler.fit_transform(data_all[cols_numeric])
    
else:
    pass

 Rank Gauss


In [14]:
# PCA
if decompo == "PCA":
    print(b_, "PCA")
    
    pca_genes = PCA(n_components = ncompo,
                    random_state = seed).fit_transform(data_all)
    
    pca_genes = pd.DataFrame(pca_genes, columns = [f"pcaHarsh-{i}" for i in range(ncompo)])
    data_all = pd.concat([data_all, pca_genes], axis=1)
else:
    pass

In [15]:
train_df = data_all[: train_df.shape[0]]
train_df = pd.concat([train_df,train_targets_scored],axis=1)
train_df.reset_index(drop = True, inplace = True)
test_df = data_all[train_df.shape[0]: ]
test_df.reset_index(drop = True, inplace = True)

train_df = train_df.drop(['index'],axis=1)
test_df = test_df.drop(['index'],axis=1)

print(f"{b_}train_df.shape: {r_}{train_df.shape}")
print(f"{b_}test_df.shape: {r_}{test_df.shape}")

X_test = test_df.values
print(f"{b_}X_test.shape: {r_}{X_test.shape}")

train_df.shape: (6337, 15)
test_df.shape: (705, 14)
X_test.shape: (705, 14)


In [16]:
def f1_eval(y_pred, dtrain):
    y_true = dtrain.get_label()
    err = accuracy_score(y_true, np.round(y_pred))
    return 'f1_err', err

In [17]:
def _F1_eval(preds, labels):
    t = np.arange(0, 1, 0.005)
    f = np.repeat(0, 200)
    results = np.vstack([t, f]).T
    # assuming labels only containing 0's and 1's
    n_pos_examples = sum(labels)
    if n_pos_examples == 0:
        raise ValueError("labels not containing positive examples")

    for i in range(200):
        pred_indexes = (preds >= results[i, 0])
        TP = sum(labels[pred_indexes])
        FP = len(labels[pred_indexes]) - TP
        precision = 0
        recall = TP / n_pos_examples

        if (FP + TP) > 0:
            precision = TP / (FP + TP)

        if (precision + recall > 0):
            F1 = 2 * precision * recall / (precision + recall)
        else:
            F1 = 0
        results[i, 1] = F1
    return (max(results[:, 1]))

def F1_eval(preds, dtrain):
    res = _F1_eval(preds, dtrain.get_label())
    return 'f1_err', 1-res

In [18]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

In [19]:
scores_auc_all = []
test_cv_preds = []

NB_SPLITS = 9

mskf = MultilabelStratifiedKFold(n_splits = NB_SPLITS, random_state = 0, shuffle = True)

oof_preds = []
oof_targets = []
scores = []
scores_auc = []


# +++++++++++++++++++++++++++++++++++++++++++++++++++++++


train_feat        = []
train_labels      = []
train_predictions = []


# +++++++++++++++++++++++++++++++++++++++++++++++++++++++++

# for mskf
ms_tar = np.array(train_df)
targets = train_df.CustomerAttrition.values
train_it = train_df.drop(['CustomerAttrition'],axis=1)
#####

for fold_nb, (train_idx, val_idx) in enumerate(mskf.split(train_it, ms_tar)):
    print(b_,"FOLDS: ", r_, fold_nb + 1)
    print(g_, '*' * 60, c_)
    
    X_train, y_train = train_it.values[train_idx, :], np.array(targets).reshape(-1,1)[train_idx]
    X_val, y_val = train_it.values[val_idx, :], np.array(targets).reshape(-1,1)[val_idx]
    ### Model ###
    
    
#     model = DecisionTreeClassifier(max_depth = 4,random_state=42)
#     model = GradientBoostingClassifier(learning_rate=0.1, min_samples_split=500,min_samples_leaf=50,max_depth=8,max_features='log2',subsample=0.9,random_state=42)
    
#     model = xgb.XGBClassifier(max_depth=6, learning_rate=0.05,silent=False, objective='binary:logistic',
#                       booster='gbtree', n_jobs=8, nthread=4, gamma=0, min_child_weight=1, max_delta_step=0,
#                       subsample=0.8, colsample_bytree=0.8, colsample_bylevel=1, reg_alpha=0, reg_lambda=1)
    model = CatBoostClassifier(iterations=5000, depth=4, learning_rate=0.04 ,l2_leaf_reg=5, loss_function='CrossEntropy',verbose=False,early_stopping_rounds = 200)
    model.fit(X_train,y_train,eval_set=(X_val, y_val),verbose_eval= False,use_best_model=True)


    out = model.predict(X_train)
    acc = accuracy_score(y_train, out)
    f1  = f1_score(y_train,out)
    print(f'Train acc: {acc} f1: {f1}')
    
    out = model.predict(X_val)
    acc = accuracy_score(y_val, out)
    f1  = f1_score(y_val,out)
    print(f'Val acc: {acc} f1: {f1}')
    
    
    ### Save OOF for CV ###
    oof_preds.append(out)
    oof_targets.append(y_val.reshape(-1,))
    scores.append(f1)
    
    ### Predict on test ###
    preds_test = model.predict_proba(X_test)[:,1:].reshape(-1)

#     preds_test = model.predict(X_test)
    test_cv_preds.append(preds_test)

oof_preds_all = np.concatenate(oof_preds)
oof_targets_all = np.concatenate(oof_targets)
test_preds_all = np.stack(test_cv_preds)

oof = accuracy_score(oof_targets_all,oof_preds_all)
print('OOF acc: %f' % oof)

sample_sub = pd.read_csv("Sample Submission.csv")

# Lets Take Weighted ensemble
base = np.zeros((len(sample_sub)))
for itr,prs in enumerate(test_preds_all):
    base += (scores[itr]/np.sum(scores))*prs

sample_sub['CustomerAttrition'] = 1*(base >= 0.5)

sample_sub.replace({1:'Yes', 0:'No'}, inplace=True)

 FOLDS:   1
 ************************************************************ 
Train acc: 0.8276229362684182 f1: 0.6266820453671665
Val acc: 0.7982954545454546 f1: 0.5477707006369426
 FOLDS:   2
 ************************************************************ 
Train acc: 0.8078835227272727 f1: 0.574685534591195
Val acc: 0.8 f1: 0.5607476635514018
 FOLDS:   3
 ************************************************************ 
Train acc: 0.8160837919403515 f1: 0.5978260869565217
Val acc: 0.8011363636363636 f1: 0.5833333333333333
 FOLDS:   4
 ************************************************************ 
Train acc: 0.8048996982069945 f1: 0.5647524752475248
Val acc: 0.7982954545454546 f1: 0.5448717948717948
 FOLDS:   5
 ************************************************************ 
Train acc: 0.8191017219953843 f1: 0.6070188970304666
Val acc: 0.7982954545454546 f1: 0.5617283950617284
 FOLDS:   6
 ************************************************************ 
Train acc: 0.8102254571276407 f1: 0.5851765618

In [20]:
dummy_t = test_df.copy()
dummy_t['CustomerAttrition'] = 1*(base >= 0.5)

In [21]:
scores_auc_all = []
test_cv_preds = []

NB_SPLITS = 9

mskf = MultilabelStratifiedKFold(n_splits = NB_SPLITS, random_state = 0, shuffle = True)

oof_preds = []
oof_targets = []
scores = []
scores_auc = []

# for mskf
ms_tar = np.array(train_df)
targets = train_df.CustomerAttrition.values
train_it = train_df.drop(['CustomerAttrition'],axis=1)
#####

for fold_nb, (train_idx, val_idx) in enumerate(mskf.split(train_it, ms_tar)):
    print(b_,"FOLDS: ", r_, fold_nb + 1)
    print(g_, '*' * 60, c_)
    
    X_train, y_train = train_it.values[train_idx, :], np.array(targets).reshape(-1,1)[train_idx]
    X_train = np.vstack([X_train,np.array(dummy_t)[:,:-1]])
    y_train = np.vstack([y_train,np.array(dummy_t)[:,-1].reshape(-1,1)])
    
    a = np.hstack([X_train,y_train])
    X_train = a[:,:-1]
    y_train = a[:,-1]
    
    X_val, y_val = train_it.values[val_idx, :], np.array(targets).reshape(-1,1)[val_idx]
    ### Model ###
    
    
#     model = DecisionTreeClassifier(max_depth = 4,random_state=42)
#     model = GradientBoostingClassifier(learning_rate=0.1, min_samples_split=500,min_samples_leaf=50,max_depth=8,max_features='log2',subsample=0.9,random_state=42)
    
#     model = xgb.XGBClassifier(max_depth=6, learning_rate=0.05,silent=False, objective='binary:logistic',
#                       booster='gbtree', n_jobs=8, nthread=4, gamma=0, min_child_weight=1, max_delta_step=0,
#                       subsample=0.8, colsample_bytree=0.8, colsample_bylevel=1, reg_alpha=0, reg_lambda=1)
    model = CatBoostClassifier(iterations=5000, depth=4, learning_rate=0.04 ,l2_leaf_reg=5, loss_function='CrossEntropy',verbose=False,early_stopping_rounds = 200)
    model.fit(X_train,y_train,eval_set=(X_val, y_val),verbose_eval= False,use_best_model=True)


#     out = model.predict(X_train)
#     acc = accuracy_score(y_train, out)
#     f1  = f1_score(y_train,out)
#     print(f'Train acc: {acc} f1: {f1}')
    
    out = model.predict(X_val)
    acc = accuracy_score(y_val, out)
    f1  = f1_score(y_val,out)
    print(f'Val acc: {acc} f1: {f1}')
    
    
    ### Save OOF for CV ###
    oof_preds.append(out)
    oof_targets.append(y_val.reshape(-1,))
    scores.append(f1)
    
    ### Predict on test ###
    preds_test = model.predict_proba(X_test)[:,1:].reshape(-1)

#     preds_test = model.predict(X_test)
    test_cv_preds.append(preds_test)

oof_preds_all = np.concatenate(oof_preds)
oof_targets_all = np.concatenate(oof_targets)
test_preds_all = np.stack(test_cv_preds)


oof = accuracy_score(oof_targets_all,oof_preds_all)
print('OOF acc: %f' % oof)

sample_sub = pd.read_csv("Sample Submission.csv")

# Lets Take Weighted ensemble
base = np.zeros((len(sample_sub)))
for itr,prs in enumerate(test_preds_all):
    base += (scores[itr]/np.sum(scores))*prs

sample_sub['CustomerAttrition'] = 1*(base >= 0.5)

sample_sub.replace({1:'Yes', 0:'No'}, inplace=True)

 FOLDS:   1
 ************************************************************ 
Val acc: 0.7954545454545454 f1: 0.5263157894736842
 FOLDS:   2
 ************************************************************ 
Val acc: 0.7957446808510639 f1: 0.5384615384615384
 FOLDS:   3
 ************************************************************ 
Val acc: 0.8082386363636364 f1: 0.599406528189911
 FOLDS:   4
 ************************************************************ 
Val acc: 0.7997159090909091 f1: 0.5495207667731629
 FOLDS:   5
 ************************************************************ 
Val acc: 0.8153409090909091 f1: 0.59375
 FOLDS:   6
 ************************************************************ 
Val acc: 0.7940340909090909 f1: 0.5510835913312694
 FOLDS:   7
 ************************************************************ 
Val acc: 0.8082386363636364 f1: 0.5871559633027522
 FOLDS:   8
 ************************************************************ 
Val acc: 0.7840909090909091 f1: 0.525
 FOLDS:   9
 ****

In [22]:
dummy_t = test_df.copy()
dummy_t['CustomerAttrition'] = base

In [23]:
scores_auc_all = []
test_cv_preds = []

NB_SPLITS = 9

mskf = MultilabelStratifiedKFold(n_splits = NB_SPLITS, random_state = 0, shuffle = True)

oof_preds = []
oof_targets = []
scores = []
scores_auc = []

# for mskf
ms_tar = np.array(train_df)
targets = train_df.CustomerAttrition.values
train_it = train_df.drop(['CustomerAttrition'],axis=1)
#####

for fold_nb, (train_idx, val_idx) in enumerate(mskf.split(train_it, ms_tar)):
    print(b_,"FOLDS: ", r_, fold_nb + 1)
    print(g_, '*' * 60, c_)
    
    X_train, y_train = train_it.values[train_idx, :], np.array(targets).reshape(-1,1)[train_idx]
    X_train = np.vstack([X_train,np.array(dummy_t)[:,:-1]])
    y_train = np.vstack([y_train,np.array(dummy_t)[:,-1].reshape(-1,1)])
    
    a = np.hstack([X_train,y_train])
    X_train = a[:,:-1]
    y_train = a[:,-1]
    
    X_val, y_val = train_it.values[val_idx, :], np.array(targets).reshape(-1,1)[val_idx]
    ### Model ###
    
    
#     model = DecisionTreeClassifier(max_depth = 4,random_state=42)
#     model = GradientBoostingClassifier(learning_rate=0.1, min_samples_split=500,min_samples_leaf=50,max_depth=8,max_features='log2',subsample=0.9,random_state=42)
    
#     model = xgb.XGBClassifier(max_depth=6, learning_rate=0.05,silent=False, objective='binary:logistic',
#                       booster='gbtree', n_jobs=8, nthread=4, gamma=0, min_child_weight=1, max_delta_step=0,
#                       subsample=0.8, colsample_bytree=0.8, colsample_bylevel=1, reg_alpha=0, reg_lambda=1)
    model = CatBoostClassifier(iterations=5000, depth=4, learning_rate=0.04 ,l2_leaf_reg=5, loss_function='CrossEntropy',verbose=False,early_stopping_rounds = 200)
    model.fit(X_train,y_train,eval_set=(X_val, y_val),verbose_eval= False,use_best_model=True)


#     out = model.predict(X_train)
#     acc = accuracy_score(y_train, out)
#     f1  = f1_score(y_train,out)
#     print(f'Train acc: {acc} f1: {f1}')
    
    out = model.predict(X_val)
    acc = accuracy_score(y_val, out)
    f1  = f1_score(y_val,out)
    print(f'Val acc: {acc} f1: {f1}')
    
    
    ### Save OOF for CV ###
    oof_preds.append(out)
    oof_targets.append(y_val.reshape(-1,))
    scores.append(f1)
    
    ### Predict on test ###
    preds_test = model.predict_proba(X_test)[:,1:].reshape(-1)

#     preds_test = model.predict(X_test)
    test_cv_preds.append(preds_test)

oof_preds_all = np.concatenate(oof_preds)
oof_targets_all = np.concatenate(oof_targets)
test_preds_all = np.stack(test_cv_preds)

 FOLDS:   1
 ************************************************************ 
Val acc: 0.796875 f1: 0.5401929260450161
 FOLDS:   2
 ************************************************************ 
Val acc: 0.8070921985815603 f1: 0.5723270440251572
 FOLDS:   3
 ************************************************************ 
Val acc: 0.7982954545454546 f1: 0.5748502994011976
 FOLDS:   4
 ************************************************************ 
Val acc: 0.8011363636363636 f1: 0.5541401273885351
 FOLDS:   5
 ************************************************************ 
Val acc: 0.8068181818181818 f1: 0.5750000000000001
 FOLDS:   6
 ************************************************************ 
Val acc: 0.8011363636363636 f1: 0.5705521472392637
 FOLDS:   7
 ************************************************************ 
Val acc: 0.8125 f1: 0.5950920245398772
 FOLDS:   8
 ************************************************************ 
Val acc: 0.7855113636363636 f1: 0.5353846153846155
 FOLDS:   9
 *

In [24]:
oof = accuracy_score(oof_targets_all,oof_preds_all)
print('OOF acc: %f' % oof)

sample_sub = pd.read_csv("Sample Submission.csv")

# Lets Take Weighted ensemble
base = np.zeros((len(sample_sub)))
for itr,prs in enumerate(test_preds_all):
    base += (scores[itr]/np.sum(scores))*prs

sample_sub['CustomerAttrition'] = 1*(base >= 0.5)

sample_sub.replace({1:'Yes', 0:'No'}, inplace=True)

dummy_t = test_df.copy()
dummy_t['CustomerAttrition'] = 1*(base >= 0.5)

OOF acc: 0.800537


In [25]:
sample_sub.to_csv("submission_3timestrain.csv",index=False)

In [29]:
sample_sub.CustomerAttrition.value_counts()

No     571
Yes    134
Name: CustomerAttrition, dtype: int64

In [36]:
np.mean(sample_sub.CustomerAttrition.values == pd.read_csv("match.csv").CustomerAttrition.values)

0.9971631205673759